In [1]:
pip install langchain

In [2]:
pip install openai

In [3]:
pip install python-dotenv


In [4]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = ['']

In [5]:
pip install pypdf

In [6]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("short-stories-for-children.pdf")
pages = loader.load()
pages = pages[4:]

In [7]:
len(pages)

40

In [8]:
page = pages[0]
print(page.page_content[0:500])

 
 
Spoken English: Short Stories  
5 
 LEVEL 1 : STORI ES FOR PRIMARY SCHOOL CHILDREN  
THE WIND AND THE SUN  
 
Once the Wind and the Sun had an argument. “I am stronger than you,” said the Wind. “No, 
you are not,” said the Sun. Just at that moment they saw a traveler walking across the road. 
He was wrapped in a s hawl. The Sun and the Wind agreed that whoever could separate the 
traveller from his shawl was stronger.  
The Wind took the first turn. He blew with all his might to tear the tra


In [9]:
page.metadata

{'source': 'short-stories-for-children.pdf', 'page': 4}

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [11]:
chunk_size = 1200
chunk_overlap = 100

In [12]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
        separators=[" "]
)

In [13]:
stories = r_splitter.split_documents(pages)
# stories

In [14]:
markdown_document = stories

In [15]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [16]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [17]:
txt = ' '.join([d.page_content for d in markdown_document])

In [18]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)
md_header_splits = markdown_splitter.split_text(txt)
# md_header_splits[0]

In [19]:
from langchain.text_splitter import TokenTextSplitter

In [20]:
pip install nltk

In [21]:
from langchain.text_splitter import NLTKTextSplitter
import nltk
nltk.download('punkt')

text_splitter = NLTKTextSplitter(chunk_size=1300)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
texts = text_splitter.split_text(txt)
# texts[0]


In [23]:
len(texts)

81

In [24]:
 pip install chromadb

In [38]:
from langchain.vectorstores import Chroma

In [39]:
persist_directory = 'docs/chroma/'

In [40]:
!rm -rf ./docs/chroma  # remove old database files if any

In [50]:
with open('document.txt', 'w') as file:
    for item in texts:
        file.write(item + '\n')

In [51]:
# import
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

In [74]:
loader = TextLoader("document.txt")
documents = loader.load()

In [76]:
text_splitter = CharacterTextSplitter(chunk_size=1200, chunk_overlap=50)
docs = text_splitter.split_documents(documents)


In [77]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [78]:
db = Chroma.from_documents(docs, embedding_function)


In [79]:
query = "hardworking"
docs = db.similarity_search(query, k=3)

In [81]:
print(docs[1].page_content)



The father smiles a nd says: "My son, pay attention."

Again the man screams: "You are a champion!"

The voice answers: "You are a champion!"

The boy is surprised, but does not understand.

Then the father explains: "People call this ECHO, but really this is LIFE.

It gives you b ack everything you say or do.

Our life is simply a reflection of our actions.

If you want more love in the world, create more love in your heart.

If you want more competence in your team, improve your competence.
Our life is simply a reflection of our actions.

If you want more love in the world, create more love in your heart.

If you want more competence in your team, improve your competence.

This relationship applies to everything, in  all aspects of life;  
Life will give you back everything you have given to it."

YOUR LIFE IS NOT A COINCIDENCE.

IT'S A REFLECTION OF YOU!"

Spoken English: Short Stories  
16 
 THE MISER  
A miser sold all that he had and bought a lump of gold, which he buried in a ho

In [82]:
# save to disk
db2 = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")
db2.persist()
docs = db.similarity_search(query)

# load from disk
db3 = Chroma(persist_directory="./chroma_db")
docs = db.similarity_search(query)
print(docs[0].page_content)

The grasshopper 
pulled a long face and walked away.

So we say "Work is real worship".

NEVER BE UNGRATEFUL  
 
It was high summer.

The sun was extremely hot.

Two travelers were going along a dusty 
road that had no trees along its sides.
So we say "Work is real worship".

NEVER BE UNGRATEFUL  
 
It was high summer.

The sun was extremely hot.

Two travelers were going along a dusty 
road that had no trees along its sides.

Looking for some shelter from the hot sun, they saw a 
tree with big leaves and branches spread like an umbrella.

shelter from the hot sun, they saw a 
tree with big leaves and branches spread like an umbrella.

They placed their b elongings on the ground and sat in the cool thick shade of the tree.

After 
taking some rest, one traveler said to the other, "What a useless tree it is!

It bears no fruits 
at all."

Hearing this, the tree felt pinched and burst out, "You ungrateful soul!

O n one hand, you are 
taking shelter in my cool shade from the burning heat

In [87]:
docs_mmr = db.max_marginal_relevance_search(query,k=3)

In [90]:
docs_mmr[1].page_content

'"There are 5 things you need to know," he told the pencil, "Before I send you out into the \nworld.\n"There are 5 things you need to know," he told the pencil, "Before I send you out into the \nworld.\n\nAlways remember them and never forget, and you will become the be st pencil you \ncan be."\n\n"One: You will be able to do many great things, but only if you allow yourself to be held in \nSomeone\'s hand."\n\n"Two: You will experience a painful sharpening from time to time, but you\'ll need it to \nbecome a better pencil."\n\n"Three: Yo u will be able to correct any mistakes you might make."\n\n"Four: The most important part of you will always be what\'s inside."\n\n"And Five: On every surface you are used on, you must leave your mark.\n\nNo matter what the \ncondition, you must continue to write."\n\nThe pencil understood and promised to remember, and went into the box with purpose in \nits heart.\n\nNow replacing the place of the pencil with you.\n\nAlways remember them and never f

In [92]:
db._collection.count()

89

In [116]:
from langchain.chat_models import ChatOpenAI
llm_name = "gpt-3.5-turbo-0301"
llm = ChatOpenAI(openai_api_key = "", model_name=llm_name, temperature=0)

In [117]:
# retrieval qa
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever()
)

In [118]:
result = qa_chain({"query": query})

RateLimitError: ignored